In [ ]:
# Install Wav2Lip and dependencies
!pip install tensorflow transformers gTTS moviepy SpeechRecognition pocketsphinx git+https://github.com/openai/whisper.git torchaudio
!git clone https://github.com/Rudrabha/Wav2Lip.git
!pip install -r Wav2Lip/requirements.txt

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-3xixfugr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-3xixfugr
  Resolved https://github.com/openai/whisper.git to commit 271445b2f24f00f8175c4fb7ae91876f7451dfc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
fatal: destination path 'Wav2Lip' already exists and is not an empty directory.
  Using cached librosa-0.7.0.tar.gz (1.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.17.1.zip (6.5 MB)
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.0.25 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59,

In [ ]:
# Download the pretrained Wav2Lip model file
!gdown --id 1-DXo5_dUP5oJWG3XBj7dyI4KfLGzhQ4d -O wav2lip_gan.pth

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1-DXo5_dUP5oJWG3XBj7dyI4KfLGzhQ4d

but Gdown can't. Please check connections and permissions.


In [ ]:
# Import necessary libraries
import moviepy.editor as mp
import whisper
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline
from gtts import gTTS
from google.colab import files
import os

In [ ]:
# Function to load translation models once
def load_translation_models(target_languages):
    translation_models = {}
    for lang in target_languages:
        model_name = f"Helsinki-NLP/opus-mt-en-{lang}"
        model = MarianMTModel.from_pretrained(model_name)
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        translation_models[lang] = (model, tokenizer)
    return translation_models

# Function to translate text using preloaded models
def translate_text(text, target_language, translation_models):
    model, tokenizer = translation_models[target_language]
    input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)
    output = model.generate(input_ids)
    translated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return translated_text


In [ ]:
# Step 1: Upload the video file
uploaded = files.upload()
video_path = list(uploaded.keys())[0]  # Get the uploaded video file name
video = mp.VideoFileClip(video_path)

# Step 2: Extract and save audio from video
audio = video.audio
audio.write_audiofile("audio.wav")


Saving test_video.mp4 to test_video.mp4
MoviePy - Writing audio in audio.wav


MoviePy - Done.


In [ ]:
# Step 3: Load and transcribe the audio file using a larger model
model = whisper.load_model("large")  # Use a larger model for better accuracy
audio_path = "audio.wav"
result = model.transcribe(audio_path)
transcribed_text = result["text"]
print("Transcribed Text: ", transcribed_text)

# Step 4: Emotion Detection (initialize once)
emotion_classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion")
emotion_result = emotion_classifier(transcribed_text)
detected_emotion = emotion_result[0]['label']
print(f"Detected Emotion: {detected_emotion}")


  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcribed Text:   Hello, my name is Ritika Shethe. My highest qualification is BMS graduate.


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

  warnings.warn(



Detected Emotion: joy


In [ ]:
# Step 5: Define target languages for translation
target_languages = ["fr", "es", "de", "hi"]  # French, Spanish, German, Hindi
translation_models = load_translation_models(target_languages)  # Load models once
translations = {}

# Step 6: Translate to each language
for lang in target_languages:
    translations[lang] = translate_text(transcribed_text, lang, translation_models)
    print(f"Translation in {lang}: {translations[lang]}")

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

Translation in fr: Bonjour, je m'appelle Ritika Shethe. Ma plus haute qualification est diplômée de BMS.
Translation in es: Hola, mi nombre es Ritika Shethe.
Translation in de: Hallo, mein Name ist Ritika Shethe. Meine höchste Qualifikation ist BMS-Absolventen.
Translation in hi: हैलो, मेरा नाम रिटिका शेर है. मेरी सबसे उच्च विशेषता BMMS स्नातक है.


In [44]:
!pip install --upgrade librosa

  Attempting uninstall: librosa
    Found existing installation: librosa 0.8.1
    Uninstalling librosa-0.8.1:
      Successfully uninstalled librosa-0.8.1


In [48]:
# Step 7: Combine translated audio with original video using Wav2Lip
for lang, translation in translations.items():
    # Adjust TTS parameters based on detected emotion
    if detected_emotion == "joy":
        tts = gTTS(text=translation, lang=lang, slow=False)  # Normal speed
    elif detected_emotion in ["sadness", "anger"]:
        tts = gTTS(text=translation, lang=lang, slow=True)  # Slower for sadness/anger
    else:
        tts = gTTS(text=translation, lang=lang, slow=False)  # Default speed

    # Generate the translated audio file for each language
    mp3_audio_path = f"translated_audio_{lang}.mp3"
    wav_audio_path = f"temp/translated_audio_{lang}.wav"
    tts.save(mp3_audio_path)
    tts.save(wav_audio_path)

    # Use Wav2Lip for lip-syncing with the specified video path
    !python /content/Wav2Lip/inference.py --checkpoint_path /content/wav2lip_gan.pth --face "{video_path}" --audio "{wav_audio_path}" --outfile "dubbed_video_{lang}.mp4"

print("Lip-synced videos created for each target language.")

Using cpu for inference.
Reading video frames...
Number of frames available for inference: 157
Traceback (most recent call last):
  File "/content/Wav2Lip/inference.py", line 280, in <module>
    main()
  File "/content/Wav2Lip/inference.py", line 225, in main
    mel = audio.melspectrogram(wav)
  File "/content/Wav2Lip/audio.py", line 47, in melspectrogram
    S = _amp_to_db(_linear_to_mel(np.abs(D))) - hp.ref_level_db
  File "/content/Wav2Lip/audio.py", line 95, in _linear_to_mel
    _mel_basis = _build_mel_basis()
  File "/content/Wav2Lip/audio.py", line 100, in _build_mel_basis
    return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
TypeError: mel() takes 0 positional arguments but 2 positional arguments (and 3 keyword-only arguments) were given
Using cpu for inference.
Reading video frames...
Number of frames available for inference: 157
Traceback (most recent call last):
  File "/content/Wav2Lip/inference.py", line 280, in <module>
    main()
  File "/content